1. 문서내용 read
2. 문서 토큰화 (쪼갬)
  - 토큰수 초과로 답변 생성하지 못할수 있다.
  - 문서가 길면(input) 답변 생성이 오래걸린다.

3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검사
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
# %pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence

In [2]:
# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/
# Recursively split과 Character Split의 차이 (\n의 숫자 및 다양한 활용)
# Recursively split >> \n을 자동으로 분류
# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/ 코드 copy url
%pip install -qU langchain-text-splitters


Note: you may need to restart the kernel to use updated packages.


In [3]:
# %pip install docx2txt

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader('../../자료/tax.docx')

document = loader.load()

len(document) #문서를 통으로 가져와서

1

텍스트 쪼개기 

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500, #토큰수
    chunk_overlap = 200, #문맥파악을 위함
)

document_list = loader.load_and_split(text_splitter=text_splitter)


In [6]:
len(document_list)

220

In [7]:
# pip install python-dotenv

In [8]:
# pip install langchain_community
# !pip install langchain-openai

In [10]:
# 쪼갠 문서를 embbeding 
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import os 

load_dotenv('../../자료/.env')
#환경 변수 파일 로드 

# 환경 변수에서 API 키 가져오기
openai_api_key = os.getenv('OPENAI_API_KEY')

# OpenAIEmbedding 최신 모델
# https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# 벡터DB에 넣을게요. Chroma DBstore에 넣을게요.
# %pip install langchain-chroma

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB ? eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached fastapi_cli-0.0.4-py3-none-any.whl.metadata (7.0 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached email_validator-2.2.0-py3-none-any.whl.metadata (25 kB)
  Using cached dnspython-2.6.1-py3-none-any.w

In [11]:
from langchain_chroma import Chroma 

database = Chroma.from_documents(documents = document_list, embedding=embedding, collection_name = 'chroma-tax'
                                 , persist_directory = "../../자료/chroma")

In [12]:
#썰, 존댓말, please하면 대답 정확도가 올라감 
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [13]:
retrieved_docs

[Document(metadata={'source': '../../자료/tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n다. 삭제<2014

[Document(metadata={'source': '../../자료/tax.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제18호 및 제21호에 따른 기타소득에 대해서는 100분의 15\n\n다. 삭제<2014. 12. 23.>\n\n라. 그 밖의 기타소득에 대해서는 100분의 20\n\n7. 퇴직소득에 대해서는 기본세율\n\n8. 대통령령으로 정하는 봉사료에 대해서는 100분의 5\n\n9. 대통령령으로 정하는 금융투자소득에 대해서는 100분의 20\n\n② 제1항에도 불구하고 다음 각 호의 이자소득 및 배당소득에 대해서는 다음 각 호에서 정하는 세율을 원천징수세율로 한다.<개정 2013. 1. 1., 2017. 12. 19., 2018. 12. 31., 2022. 12. 31.>\n\n1. 「민사집행법」 제113조 및 같은 법 제142조에 따라 법원에 납부한 보증금 및 경락대금에서 발생하는 이자소득에 대해서는 100분의 14\n\n2. 대통령령으로 정하는 실지명의가 확인되지 아니하는 소득에 대해서는 100분의 45. 다만, 「금융실명거래 및 비밀보장에 관한 법률」 제5조가 적용되는 경우에는 같은 조에서 정한 세율로 한다.'),
 Document(metadata={'source': '../../자료/tax.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 요양보상금, 휴업보상금, 상병보상금(傷病補償金), 일시보상금, 장해보상금, 유족보상금, 행방불명보상금, 소지품 유실보상금, 장의비 및 장제비\n\n마. 「고용보험법」에 따라 받는 실업급여, 육아휴직 급여, 육아기 근로시간 단축 급여, 출산전후휴가 급여등, 「제대군인 지원에 관한 법률」에 따라 받는 전직지원금, 「국가공무원법」ㆍ「지방공무원법」에 따른 공무원 또는 「사립학교교직원 연금법」ㆍ「별정우체국법」을 적용받는 사람이 관련 법령에 따라 받는 육아휴직수당(「사립학교법」 제70조의2에 따라 임명된 사무직원이 학교의 정관 또는 규칙에 따라 지급받는 육아휴직수당으로서 대통령령으로 정하는 금액 이하의 것을 포함한다)\n\n바. 「국민연금법」에 따라 받는 반환일시금(사망으로 받는 것만 해당한다) 및 사망일시금'),
 Document(metadata={'source': '../../자료/tax.docx'}, page_content='서. 「교육기본법」 제28조제1항에 따라 받는 장학금 중 대학생이 근로를 대가로 지급받는 장학금(「고등교육법」 제2조제1호부터 제4호까지의 규정에 따른 대학에 재학하는 대학생에 한정한다)\n\n어. 「발명진흥법」 제2조제2호에 따른 직무발명으로 받는 다음의 보상금(이하 “직무발명보상금”이라 한다)으로서 대통령령으로 정하는 금액\n\n\u3000\u3000\u30001) 「발명진흥법」 제2조제2호에 따른 종업원등(이하 이 조, 제20조 및 제21조에서 “종업원등”이라 한다)이 같은 호에 따른 사용자등(이하 이 조에서 “사용자등”이라 한다)으로부터 받는 보상금. 다만, 보상금을 지급한 사용자등과 대통령령으로 정하는 특수관계에 있는 자가 받는 보상금은 제외한다.\n\n\u3000\u3000\u30002) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법 제32조제1항제4호에 따라 받는 보상금\n\n저. 대통령령으로 정하는 복리후생적 성질의 급여\n\n4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 「국민연금법」, 「공무원연금법」 또는 「공무원 재해보상법」, 「군인연금법」 또는 「군인 재해보상법」, 「사립학교교직원 연금법」, 「별정우체국법」 또는 「국민연금과 직역연금의 연계에 관한 법률」(이하 “공적연금 관련법”이라 한다)에 따라 받는 유족연금ㆍ퇴직유족연금ㆍ퇴역유족연금ㆍ장해유족연금ㆍ상이유족연금ㆍ순직유족연금ㆍ직무상유족연금ㆍ위험직무순직유족연금, 장애연금, 장해연금ㆍ비공무상 장해연금ㆍ비직무상 장해연금, 상이연금(傷痍年金), 연계노령유족연금 또는 연계퇴직유족연금\n\n나. 삭제<2013. 1. 1.>\n\n다. 「산업재해보상보험법」에 따라 받는 각종 연금\n\n라. 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금\n\n마. 삭제<2013. 1. 1.>\n\n5. 기타소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품\n\n나. 「국가보안법」에 따라 받는 상금과 보로금\n\n다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상\n\n라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.\n\n마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품\n\n바. 「문화유산의 보존 및 활용에 관한 법률」에 따라 국가지정문화유산으로 지정된 서화ㆍ골동품의 양도로 발생하는 소득')]

In [14]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o')

In [15]:
#페르소나 주기
prompt = f'''[Idendtity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.
[Context]
{retrieved_docs}

Question: {query}
'''

In [ ]:
ai_message = llm.invoke(prompt)

In [ ]:
ai_message.content

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. 기본세율은 근로소득에 적용되는 세율로, 다음과 같은 누진세율 구조를 가지고 있습니다. \n\n2023년 기준 한국의 소득세율 표는 다음과 같습니다:\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억5천만원 이하: 35%\n- 1억5천만원 초과 ~ 3억원 이하: 38%\n- 3억원 초과 ~ 5억원 이하: 40%\n- 5억원 초과: 42%\n\n연봉 5천만원의 경우, 기본적으로 연말정산 과정에서 공제항목들이 반영되기 때문에 정확한 세액은 다를 수 있지만, 여기서는 단순화를 위해 기본적인 소득세율만을 적용해 보겠습니다.\n\n1. 1,200만원까지는 6% 세율 적용:\n   \\[\n   1,200만원 \\times 0.06 = 72만원\n   \\]\n\n2. 1,200만원 초과 ~ 4,600만원까지는 15% 세율 적용:\n   \\[\n   (4,600만원 - 1,200만원) \\times 0.15 = 3,400만원 \\times 0.15 = 510만원\n   \\]\n\n3. 4,600만원 초과 ~ 5,000만원까지는 24% 세율 적용:\n   \\[\n   (5,000만원 - 4,600만원) \\times 0.24 = 400만원 \\times 0.24 = 96만원\n   \\]\n\n따라서 총 소득세는:\n\\[\n72만원 + 510만원 + 96만원 = 678만원\n\\]\n\n이 계산은 기본적인 세율만을 반영한 것이며, 실제 소득세액은 각종 공제 (예: 인적공제, 보험료 공제, 연금보험료 공제 등)를 반영한 후에 결정됩니다. 연말정산 시 공제항목들을 반영하면 실제 부담하는 세액은 이보다 적을 가능성이 높습니다.'

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. 기본세율은 근로소득에 적용되는 세율로, 다음과 같은 누진세율 구조를 가지고 있습니다. \n\n2023년 기준 한국의 소득세율 표는 다음과 같습니다:\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억5천만원 이하: 35%\n- 1억5천만원 초과 ~ 3억원 이하: 38%\n- 3억원 초과 ~ 5억원 이하: 40%\n- 5억원 초과: 42%\n\n연봉 5천만원의 경우, 기본적으로 연말정산 과정에서 공제항목들이 반영되기 때문에 정확한 세액은 다를 수 있지만, 여기서는 단순화를 위해 기본적인 소득세율만을 적용해 보겠습니다.\n\n1. 1,200만원까지는 6% 세율 적용:\n   \\[\n   1,200만원 \\times 0.06 = 72만원\n   \\]\n\n2. 1,200만원 초과 ~ 4,600만원까지는 15% 세율 적용:\n   \\[\n   (4,600만원 - 1,200만원) \\times 0.15 = 3,400만원 \\times 0.15 = 510만원\n   \\]\n\n3. 4,600만원 초과 ~ 5,000만원까지는 24% 세율 적용:\n   \\[\n   (5,000만원 - 4,600만원) \\times 0.24 = 400만원 \\times 0.24 = 96만원\n   \\]\n\n따라서 총 소득세는:\n\\[\n72만원 + 510만원 + 96만원 = 678만원\n\\]\n\n이 계산은 기본적인 세율만을 반영한 것이며, 실제 소득세액은 각종 공제 (예: 인적공제, 보험료 공제, 연금보험료 공제 등)를 반영한 후에 결정됩니다. 연말정산 시 공제항목들을 반영하면 실제 부담하는 세액은 이보다 적을 가능성이 높습니다.'

In [ ]:
# %pip install -U langchain langchainhub --quiet
# 프롬프트를 준다

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [16]:
# 프롬프트 추천
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')

In [ ]:
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {'prompt' : prompt}
)

In [20]:
ai_message = qa_chain({'query':query}) #전처리를 더 해야함 
ai_message

d:\MulCam\13_LangChain\.venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 기본세율에 따라 계산됩니다. 기본세율은 과세표준에 따라 다르므로, 정확한 금액은 소득공제 등을 고려한 후 결정됩니다. 기본세율 적용 방식에 대한 자세한 계산이 필요합니다.'}